# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233848681599                   -0.50    8.0
  2   -7.250267252485       -1.78       -1.40    1.0
  3   -7.251307244810       -2.98       -2.11    4.0
  4   -7.251281149813   +   -4.58       -2.22    4.0
  5   -7.251330416818       -4.31       -2.70    3.0
  6   -7.251337059783       -5.18       -3.02    2.0
  7   -7.251338765993       -5.77       -3.91    1.0
  8   -7.251338793980       -7.55       -4.22    4.0
  9   -7.251338798377       -8.36       -4.81    2.0
 10   -7.251338798688       -9.51       -5.46    3.0
 11   -7.251338798703      -10.81       -5.96    3.0
 12   -7.251338798704      -11.95       -6.35    3.0
 13   -7.251338798705      -12.68       -7.04    3.0
 14   -7.251338798705      -14.10       -7.48    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06530975895000443
[ Info: Arnoldi iteration step 2: normres = 0.3264804049614061
[ Info: Arnoldi iteration step 3: normres = 1.0370480216576365
[ Info: Arnoldi iteration step 4: normres = 0.3704149415348673
[ Info: Arnoldi iteration step 5: normres = 0.6138526548895391
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.03e-01, 8.61e-02, 5.73e-01, 1.75e-01, 1.05e-02)
[ Info: Arnoldi iteration step 6: normres = 0.20356538725295625
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.57e-02, 7.72e-02, 1.31e-01, 1.06e-01, 3.51e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09250930703968632
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.95e-04, 6.68e-03, 1.06e-02, 5.82e-02, 6.44e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13339798684956097
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.05e-